<a href="https://colab.research.google.com/github/SofiaCR2/Python-basico-intermedio/blob/main/ch15_Sobrecarga_polimorfismo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Más programación orientada a objetos, sobrecarga de operadores y polimorfismo

http://openbookproject.net/thinkcs/python/english3e/even_more_oop.html

## Mi tiempo
- clase que registra la hora del día
- proporcione el método __init__ para que cada instancia se cree con los atributos y la inicialización apropiados

In [1]:
class MyTime:
    """MyTime class that keeps track of time of day"""
    def __init__(self, hrs=0, mins=0, secs=0):
        """ Creates a MyTime object initialized to hrs, mins, secs """
        self.hours = hrs
        self.minutes = mins
        self.seconds = secs

    def __str__(self):
        return "{:02}:{:02}:{:02}".format(self.hours, self.minutes, self.seconds)

In [2]:
tim1 = MyTime(11, 59, 3)

In [3]:
print(tim1)

11:59:03


## Las funciones pueden ser puras y modificadoras
- ¿Qué funciones deberían ser parte de la clase de métodos?
- por lo general, todas las funciones que operan o usan atributos de clase deben ser parte de la clase llamada métodos

## funciones puras
- las funciones puras no tienen efectos secundarios, como modificar parámetros y variables globales
- similar a las funciones constantes en el mundo de C++
- los métodos getter son funciones puras
- por ejemplo: ver add_time()

In [5]:
def add_time(t1, t2):
    h = t1.hours + t2.hours
    m = t1.minutes + t2.minutes
    s = t1.seconds + t2.seconds

    while s >= 60:
        s -= 60
        m += 1

    while m >= 60:
        m -= 60
        h += 1

    sum_t = MyTime(h, m, s)
    return sum_t

In [6]:
current_time = MyTime(9, 50, 45)
bread_time = MyTime(2, 35, 20)
done_time = add_time(current_time, bread_time)
print(done_time)

12:26:05


## modificadores
- funciones que modifican el(los) objeto(s) que obtiene como parámetro(s)
- los métodos setter son modificadores

In [7]:
# function takes MyTime t and secs to update t
def increment(myT, seconds):
    myT.seconds += seconds
    mins = myT.seconds//60

    myT.seconds = myT.seconds%60
    myT.minutes += mins

    hours = myT.minutes//60
    myT.hours += hours
    myT.minutes = myT.minutes%60

In [8]:
current_time = MyTime(9, 50, 45)
print(current_time)

09:50:45


In [9]:
increment(current_time, 60*60)

In [10]:
print(current_time)

10:50:45


## Convirtiendo increment() en un método
- OOD prefiere que las funciones que trabajan con objetos sean parte de la clase o métodos
- el incremento puede ser un método útil para la clase MyTime

In [11]:
class MyTime:
    def __init__(self, hrs=0, mins=0, secs=0):
        """ Create a new MyTime object initialized to hrs, mins, secs.
           The values of mins and secs may be outside the range 0-59,
           but the resulting MyTime object will be normalized.
        """
        self.hours = hrs
        self.minutes = mins
        self.seconds = secs

        # Calculate total seconds to represent
        self.__normalize()

    def __str__(self):
        return "{:02}:{:02}:{:02}".format(self.hours, self.minutes, self.seconds)

    def to_seconds(self):
        """ Return the number of seconds represented
            by this instance
        """
        return self.hours * 3600 + self.minutes * 60 + self.seconds

    def increment(self, seconds):
        self.seconds += seconds
        self.__normalize()

    # should be treated as private method
    def __normalize(self):
        totalsecs = self.to_seconds()
        self.hours = totalsecs // 3600        # Split in h, m, s
        leftoversecs = totalsecs % 3600
        self.minutes = leftoversecs // 60
        self.seconds = leftoversecs % 60

In [12]:
# improved add_time function
def add_time(t1, t2):
    secs = t1.to_seconds() + t2.to_seconds()
    return MyTime(0, 0, secs)

In [13]:
# test add_time function
current_time = MyTime(9, 50, 45)
bread_time = MyTime(2, 35, 20)
done_time = add_time(current_time, bread_time)
print(done_time)

12:26:05


### De manera similar, add_time se puede mover dentro de la clase MyTime como método

In [14]:
class MyTime:
    def __init__(self, hrs=0, mins=0, secs=0):
        """ Create a new MyTime object initialized to hrs, mins, secs.
           The values of mins and secs may be outside the range 0-59,
           but the resulting MyTime object will be normalized.
        """
        self.hours = hrs
        self.minutes = mins
        self.seconds = secs
        # Calculate total seconds to represent
        self.__normalize()

    def __str__(self):
        return "{:02}:{:02}:{:02}".format(self.hours, self.minutes, self.seconds)

    def to_seconds(self):
        """ Return the number of seconds represented
            by this instance
        """
        return self.hours * 3600 + self.minutes * 60 + self.seconds

    def increment(self, secs):
        self.seconds += secs
        self.__normalize()

    def __normalize(self):
        totalsecs = self.to_seconds()
        self.hours = totalsecs // 3600        # Split in h, m, s
        leftoversecs = totalsecs % 3600
        self.minutes = leftoversecs // 60
        self.seconds = leftoversecs % 60

    def add_time(self, other):
        return MyTime(0, 0, self.to_seconds() + other.to_seconds())


In [15]:
current_time = MyTime(9, 50, 45)
bread_time = MyTime(2, 35, 20)
done_time = current_time.add_time(bread_time)
print(done_time)

12:26:05


## métodos especiales / sobrecarga de operadores
- https://docs.python.org/3/reference/datamodel.html
- ¿Qué tal t1 = t2 + t3 al igual que agregar tipos primitivos?
- El operador \+ agrega dos cadenas, pero agrega dos enteros o flotantes
- el mismo operador tiene un significado diferente para diferentes tipos llamado sobrecarga de operadores
- reemplace add_time con el método especial incorporado __add__ para sobrecargar + operador

In [16]:
class MyTime:

    def __init__(self, hrs=0, mins=0, secs=0):
        """ Create a new MyTime object initialized to hrs, mins, secs.
           The values of mins and secs may be outside the range 0-59,
           but the resulting MyTime object will be normalized.
        """
        self.hours = hrs
        self.minutes = mins
        self.seconds = secs
        # Calculate total seconds to represent
        self.__normalize()

    def __str__(self):
        return "{:02}:{:02}:{:02}".format(self.hours, self.minutes, self.seconds)

    def to_seconds(self):
        """ Return the number of seconds represented
            by this instance
        """
        return self.hours * 3600 + self.minutes * 60 + self.seconds

    def increment(self, secs):
        self.seconds += secs
        self.normalize()

    def __normalize(self):
        totalsecs = self.to_seconds()
        self.hours = totalsecs // 3600        # Split in h, m, s
        leftoversecs = totalsecs % 3600
        self.minutes = leftoversecs // 60
        self.seconds = leftoversecs % 60

    def __add__(self, other):
        return MyTime(0, 0, self.to_seconds() + other.to_seconds())

In [17]:
current_time = MyTime(9, 50, 45)
bread_time = MyTime(2, 35, 20)
done_time = current_time + bread_time # equivalent to: done_time = current_time.__add__(bread_time)
print(done_time)

12:26:05


## agregar dos puntos
- sobrecargar nuestra clase Point para poder sumar dos puntos

In [18]:
class Point:
    """
    Point class represents and manipulates x,y coords
    """
    count = 0

    def __init__(self, xx=0, yy=0):
        """Create a new point with given x and y coords"""
        self.x = xx
        self.y = yy
        Point.count += 1

    def dist_from_origin(self):
        import math
        dist = math.sqrt(self.x**2+self.y**2)
        return dist

    def __str__(self):
        return "({}, {})".format(self.x, self.y)

    def move(self, xx, yy):
        self.x = xx
        self.y = yy

    def __add__(self, other):
        x = self.x + other.x
        y = self.y + other.y
        return Point(x, y)

    def __mul__(self, other):
        """
        computes dot product of two points
        """
        return self.x * other.x + self.y * other.y

    def __rmul__(self, other):
        """
        if the left operand is primitive type (int or float)
        and the right operand is a Point, Python invokes __rmul__
        which performs scalar multiplication
        """
        return Point(other * self.x, other * self.y)


In [19]:
p1 = Point(2, 2)
p2 = Point(10, 10)
p3 = p1 + p2
print(p3)
print(p1 * p3)
print(4 * p1)

(12, 12)
48
(8, 8)


## some special methods
<pre>
__del__(self)
    - destructor - called when instance is about to be destroyed
    
__str__(self)
   - called by str(object) and the built-in functions format() and print() to computer the "informal" or nicely printable string representation of an object.
   - must return string object

__lt__(self, other)
    x &lt; y calls x.__lt__(y)

__gt__(self, other)
    x &gt; y calls x.__gt__(y)
   
__eq__(self, other)
    x == y calls x.__eq__(y)

__ne__(self, other)
__ge__(self, other)
__le__(self, other)

Emulating numeric types:
__add__(self, other)
__sub__(self, other)
__mul__(self, other)
__mod__(self, other)
__truediv__(self, other)
__pow__(self, other)
__xor__(self, other)
__or__(self, other)
__and__(self, other)
</pre>

## Polimorfismo
- la mayoría de los métodos funcionan en un nuevo tipo de clase específico que creamos
- algunos métodos que queremos aplicar a muchos tipos, como operaciones aritméticas + en el ejemplo anterior
- por ejemplo, la operación multadd (común en álgebra lineal) toma 3 argumentos, multiplica los dos primeros y luego suma el tercero
- función como esta que puede tomar argumentos con diferentes tipos se llama polimórfica

In [20]:
def multadd(x, y, z):
    return x * y + z

In [21]:
multadd(3, 2, 1)

7

In [22]:
p1 = Point(3, 4)
p2 = Point(5, 7)
print(multadd(2, p1, p2))

(11, 15)


In [23]:
print(multadd (p1, p2, 1))

44


## regla de escritura pato - enlace dinámico
- prueba del pato: "Si camina como un pato y grazna como un pato, entonces debe ser un pato"
- para determinar si una función se puede aplicar a un nuevo tipo, aplicamos la regla fundamental de polimorfismo de Python, llamada regla de tipeo de pato: <em> si todas las operaciones dentro de la función se pueden aplicar al tipo, la función se puede aplicar al tipo </em>
- por ejemplo: https://en.wikipedia.org/wiki/Duck_typing

In [25]:
class Duck:
    def fly(self):
        print("Duck flying")

class Airplane:
    def fly(self):
        print("Airplane flying")

class Whale:
    def swim(self):
        print("Whale swimming")

# polymorphism
def lift_off(entity):
    entity.fly()
    # ¡solo arroja un error si alguna entidad no tiene un atributo de vuelo durante el tiempo de ejecución!
    # ¡Los lenguajes escritos estáticamente como C++ dan errores de tiempo de compilación!

duck = Duck() # objeto
airplane = Airplane()
#whale = Whale()

lift_off(duck) # prints `Duck flying`
lift_off(airplane) # prints `Airplane flying`
#lift_off(whale) # Throws the error `'Whale' object has no attribute 'fly'`

Duck flying
Airplane flying


## Acciones que tienen el mismo nombre pero tiene diferente funcion o actividad (esto es por herencia)

In [29]:
class Animal:
  def __init__(self, nombre):
    self.nombre = nombre

  def tipo_animal(self):
    pass

class Leon(Animal):
  def tipo_animal(self):
    print('animal salvaje')

class Perro(Animal):
  def tipo_animal(self):
    print('animal domestico')

nuevo_animal = Leon('simba')
nuevo_animal.tipo_animal()

nuevo_animal2 = Perro('Niik')
nuevo_animal2.tipo_animal()

animal salvaje
animal domestico
